In [2]:
import pandas as pd
import numpy as np
from anndata import read_h5ad, AnnData
import scanpy as sc

In [3]:
metmap_tissue = snakemake.params['metmap_tissue']

In [4]:
ccle_adata = read_h5ad(snakemake.input['ccle_exp'])

In [5]:
mm_all_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.all5", index_col=0)
mm_tissue_df = pd.read_excel(snakemake.input['mm_potential'], sheet_name=f"metp500.{metmap_tissue}", index_col=0)

In [6]:
ccle_adata.obs

In [7]:
mm_tissue_df.head()

In [9]:
mm_celllines = mm_tissue_df.index.values.tolist()
cellline_intersect = set(ccle_adata.obs.index.values.tolist()).intersection(set(mm_celllines))
len(cellline_intersect)

In [11]:
ccle_adata.obs = ccle_adata.obs.merge(mm_tissue_df, how="left", left_index=True, right_index=True)

In [13]:
mm_rows = ccle_adata.obs.index.to_series().apply(lambda cid: cid in cellline_intersect)
ccle_adata = ccle_adata[mm_rows, :]

In [16]:
ccle_adata.obs["metastatic"] = ccle_adata.obs["penetrance"].apply(lambda p: p > 0.0)

In [17]:
ccle_adata.obs

In [18]:
counts_df = pd.DataFrame(data=ccle_adata.X, columns=ccle_adata.var.index.values.tolist(), index=ccle_adata.obs.index.values.tolist())

In [20]:
conditions_df = ccle_adata.obs[["mean", "penetrance", "metastatic"]]

In [21]:
counts_df.to_csv(snakemake.output["counts"])
conditions_df.to_csv(snakemake.output["conditions"])